In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
from tqdm import tqdm
tqdm.pandas()

import os
import sys
sys.path.append('../scripts/')
from analyze_sentiments import analyze_sentiments

In [2]:
rs_path = glob('../data/Reddit Submissions/*.csv')
# print(rs_path)

In [3]:
# concatenate all the csv files into one dataframe
rs = pd.concat([pd.read_csv(f) for f in rs_path], ignore_index=True)
print(rs.shape)
rs.head(10)

(141526, 5)


,subreddit,title,selftext,sector,created_utc
0,personalfinance,How to navigate the IRS Federal Withholding Ca...,"Hi PF,\n\nI'm looking at the IRS Withholding c...",finance,1.483232e+09
1,Economics,"Complexity, not size, is the real danger in ba...",[deleted],finance,1.483236e+09
2,Economics,"Complexity, not size, is the real danger in ba...",[deleted],finance,1.483237e+09
3,Economics,"Complexity, not size, is the real danger in ba...",[deleted],finance,1.483237e+09
4,personalfinance,Any way to get out of a car lease that was sig...,So my aunt fell prey to a predatory lendor it ...,finance,1.483237e+09
5,Economics,"Complexity, not size, is the real danger in ba...",NaN,finance,1.483237e+09
6,personalfinance,Amex Blue Cash Preferred or EveryDay after Sal...,I have a QuickSilver and the Sallie Mae BC Mas...,finance,1.483240e+09
7,personalfinance,Im nearly back in control but need some advice,"Hi all, I've been subscribed to this subreddit...",finance,1.483246e+09
8,personalfinance,Vacation or no vacation?,We've had some 3 day Disneyland tickets gifted...,finance,1.483247e+09
9,economy,"Complexity, not size, is the real danger in ba...",NaN,finance,1.483250e+09


In [4]:
# convert the 'created_utc' column to datetime YYYY-MM-DD format
rs['created_utc'] = pd.to_datetime(rs['created_utc'], unit='s').dt.strftime('%Y-%m-%d')

# sort the dataframe by 'created_utc' column
rs = rs.sort_values(by='created_utc', ascending=True)
rs = rs.reset_index(drop=True)

rs.head(10)

,subreddit,title,selftext,sector,created_utc
0,personalfinance,How to navigate the IRS Federal Withholding Ca...,"Hi PF,\n\nI'm looking at the IRS Withholding c...",finance,2017-01-01
1,economy,Financial Times: 2016 in 14 charts – Financial...,NaN,finance,2017-01-01
2,personalfinance,"How to tackle student loans, about to be a Fre...",Hello r/personalfinanace\n\nAbout a month ago ...,finance,2017-01-01
3,personalfinance,I raised my credit score by 73 points in 3 mon...,I know I still have work to do but I wanted to...,finance,2017-01-01
4,personalfinance,"I got stiffed for a job, and I need advice.",I am a marine mechanic working side jobs. I li...,finance,2017-01-01
5,personalfinance,Need advice for first time ISA (UK),"Hi guys, just looking for advice really - I am...",finance,2017-01-01
6,personalfinance,I'm 18 years old and bought myself a 50k car -...,"sup reddit, created a replacement acc because ...",finance,2017-01-01
7,personalfinance,Is my Chase Mortgage Broker Lying?,Hey PF! I've been a long time lurker and I'm a...,finance,2017-01-01
8,personalfinance,Student Loans - Car Security,Hi \n\nIs it possible to use the equity in our...,finance,2017-01-01
9,personalfinance,Wife trying to take more control of retirement,My wife (28) is trying to take more control of...,finance,2017-01-01


In [5]:
# apply sentiment analysis to the 'title' + 'selftext' columns
rs = rs.fillna('')

sentiments = []
batch_size = 50
for i in tqdm(range(0, len(rs), batch_size)):
    sentiments += analyze_sentiments(rs['title'].iloc[i:i+batch_size] + ' ' + rs['selftext'].iloc[i:i+batch_size], mode='agg')

rs['sentiment'] = sentiments
rs.to_csv('../data/results/reddit_submissions_sentiments.csv', index=False)

100%|██████████| 2831/2831 [7:37:32<00:00,  9.70s/it]  
